In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import os

import pandas as pd
import numpy as np
import open3d as o3d

from src import Config
from src.common.logger import ILogger, Logger
from src.feature import Preprocess
from src.feature.preprocess.data_dto import DataDto
from src.model import AutoGluon
from src.feature.display import Visualize

## Reading train and test data from csv

In [25]:
train_data: pd.DataFrame = pd.read_csv(os.path.join(Config.DATASETS_DIR.value, 'labled_dataset.csv'))
test_data: pd.DataFrame = pd.read_csv(os.path.join(Config.DATASETS_DIR.value, 'raw_point_cloud.csv'))

In [26]:
raw_data: DataDto = DataDto(train_data, test_data)

## Preprocessing

In [27]:
logger: ILogger = Logger()
preprocess: Preprocess = Preprocess(dto=raw_data, logger=logger)

## AutoGluon predictor

In [28]:
auto_gluon: AutoGluon = AutoGluon(
    train_dto=preprocess.train_dto,
    test_dto=preprocess.test_dto,
    logger=logger,
    train_max_minutes=300
)

### Training

In [29]:
auto_gluon.fit()

INFO:src.common.logger.logger:A model has already been trained and saved at C:\Users\jatha\Code\NTNU\TBA4256-3D-Digital-Modelling\ScangelinaJolieV2\models\auto_gluon\ag_model_300
INFO:src.common.logger.logger:Loading model.
INFO:src.common.logger.logger:Model loaded.


### Predicting

In [30]:
auto_gluon.predict()

INFO:src.common.logger.logger:Loading predictions from C:\Users\jatha\Code\NTNU\TBA4256-3D-Digital-Modelling\ScangelinaJolieV2\data\output\predictions\prediction_300.csv


## Building voxel grid

In [31]:
triangle_mesh: o3d.geometry.TriangleMesh = o3d.geometry.TriangleMesh()
test_data: pd.DataFrame = pd.merge(
    auto_gluon.test_dto.bounding_box,
    auto_gluon.test_dto.voxel_color,
    left_index=True,
    right_index=True
)

In [32]:
def create_bounding_box(mesh: o3d.geometry.TriangleMesh, row: np.ndarray) -> None:
    max_bound: np.ndarray = row[0:3]
    min_bound: np.ndarray = row[3:6]
    color: np.ndarray = row[6:9]

    box: o3d.geometry.TriangleMesh = o3d.geometry.TriangleMesh.create_box(
        width=max_bound[0] - min_bound[0],
        height=max_bound[1] - min_bound[1],
        depth=max_bound[2] - min_bound[2]
    )

    box.translate(min_bound)
    box.paint_uniform_color(color)
    mesh += box
    
    return row

In [33]:
test_data.apply(lambda row: create_bounding_box(triangle_mesh, row), axis=1)

,max_bound_x,max_bound_y,max_bound_z,min_bound_x,min_bound_y,min_bound_z,r,g,b
0,570090.7,7034137.29,31.98,570089.7,7034136.29,30.98,0.419422,0.481344,0.303245
1,570062.7,7034225.29,3.98,570061.7,7034224.29,2.98,0.787121,0.725597,0.639658
2,570121.7,7034378.29,9.98,570120.7,7034377.29,8.98,0.305425,0.248539,0.233890
3,570035.7,7034202.29,11.98,570034.7,7034201.29,10.98,0.545818,0.452540,0.384239
4,569991.7,7034339.29,12.98,569990.7,7034338.29,11.98,0.634189,0.596103,0.500984
...,...,...,...,...,...,...,...,...,...
294849,569804.7,7034358.29,27.98,569803.7,7034357.29,26.98,0.641769,0.650212,0.619969
294850,569855.7,7034150.29,22.98,569854.7,7034149.29,21.98,0.300972,0.313435,0.307110
294851,570071.7,7034188.29,6.98,570070.7,7034187.29,5.98,0.477578,0.464221,0.442043
294852,570248.7,7034176.29,35.98,570247.7,7034175.29,34.98,0.243754,0.219378,0.227503


In [34]:
Visualize.mesh(triangle_mesh)

In [35]:
def save_grid(mesh: o3d.geometry.TriangleMesh, filename: str) -> None:
        path: str = os.path.join(Config.VOXEL_GRID_DIR.value, f'{filename}.ply')
        o3d.io.write_triangle_mesh(filename=path, mesh=mesh)

In [36]:
save_grid(triangle_mesh, 'unlabled_voxel_grid')